# Staging OpenAI API


In [1]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")

client = OpenAI(api_key=api_key)

In [2]:
user_input = "Hello, suggest a trip plan in Tokyo"

messages = [
    {"role": "system", "content": "Only suggest 1 activity for the trip plane"},
    {"role": "user", "content": user_input},
]

completion = client.chat.completions.create(model="gpt-4o-mini", messages=messages)
re_json = completion.choices[0].message.content
print(re_json)

For your trip to Tokyo, I recommend visiting the iconic Shibuya Crossing. It's one of the busiest pedestrian intersections in the world and offers a vibrant atmosphere that embodies the spirit of Tokyo. You can explore the surrounding area, check out the Hachiko statue, and enjoy shopping at nearby stores or grab a meal at one of the many restaurants. Be sure to experience the view from a nearby café or the Shibuya Sky observation deck for a breathtaking perspective of the city!


# GMAIL ROUTE


In [39]:
import base64


# GPT written, just decodes gmail body content data
def decode_base64_data(data_str):
    missing_padding = len(data_str) % 4
    if missing_padding:
        data_str += "=" * (4 - missing_padding)
    decoded_bytes = base64.urlsafe_b64decode(data_str.encode("utf-8"))
    return decoded_bytes.decode("utf-8", errors="replace")


import os.path
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build


def get_credentials(token_path="token.json", scopes=None):
    """
    Gets credentials from user
    Args:
        token_path: (str) the token.json file unique to local users
        scopes: (str) sets scope for get_credentials
    """
    if scopes is None:
        scopes = ["https://www.googleapis.com/auth/gmail.readonly"]
    creds = None
    if os.path.exists(token_path):
        creds = Credentials.from_authorized_user_file(token_path, scopes)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            client_config = {
                "installed": {
                    "client_id": os.environ.get("GOOGLE_CLIENT_ID_DESK"),
                    "client_secret": os.environ.get("GOOGLE_CLIENT_SECRET_DESK"),
                    "auth_uri": "https://accounts.google.com/o/oauth2/auth",
                    "token_uri": "https://oauth2.googleapis.com/token",
                }
            }
            flow = InstalledAppFlow.from_client_config(client_config, scopes)
            creds = flow.run_local_server(port=0)
        with open(token_path, "w") as token_file:
            token_file.write(creds.to_json())
    return creds


def build_gmail_service(creds):
    return build("gmail", "v1", credentials=creds)

In [58]:
# gets credentials
creds = get_credentials()

# build the gmail service
service = build_gmail_service(creds)

# Id's for messages
results = (
    service.users().messages().list(userId="me", q="is:unread").execute()["messages"]
)

# using getting the top 10 messages
top_k_messages = results[:10] if len(results) > 10 else results

# getting the each message with ids
message_ids = [message.get("id") for message in top_k_messages]

# getting the resulting messages
res_messages = [
    service.users()
    .messages()
    .get(userId="me", id=id, format="full")
    .execute()
    .get("payload")
    for id in message_ids
]

## Overview of return object from GMAIL API

How to navigate this json object


In [59]:
from pprint import pprint

pprint(res_messages[0])

{'body': {'size': 0},
 'filename': '',
 'headers': [{'name': 'Delivered-To', 'value': 'contact@vizcom.co'},
             {'name': 'Received',
              'value': 'by 2002:a05:7022:4199:b0:9a:c19:4c0e with SMTP id '
                       'i25csp4903024dli;        Tue, 27 May 2025 11:39:08 '
                       '-0700 (PDT)'},
             {'name': 'X-Received',
              'value': 'by 2002:a05:6e02:2482:b0:3d3:db70:b585 with SMTP id '
                       'e9e14a558f8ab-3dc9b72443emr160912575ab.21.1748371148703;        '
                       'Tue, 27 May 2025 11:39:08 -0700 (PDT)'},
             {'name': 'ARC-Seal',
              'value': 'i=1; a=rsa-sha256; t=1748371148; cv=none;        '
                       'd=google.com; s=arc-20240605;        '
                       'b=isynD+b6SuUAWfDbkolPg0eyOZRvXQKM5qggr55/5pBe4lK3FavGBfPrtbaI73687r         '
                       'xX5YUNJR4LflVygWUr5JcRZ2D8RmNqQ0Sx3q1PRAghsqLU7NVfHU9Bg/ElH09Av98I1l         '
                   

## Retrieving Vital Information


In [78]:
# getting subject and subject
sender, subject = "", ""

for header in res_messages[0]["headers"]:
    if header["name"] == "Subject":
        subject = header.get("value")
    elif header["name"] == "From":
        sender = header.get("value")

# getting
data = res_messages[0]["parts"][0]["body"]["data"]

### Normalizing Data


In [79]:
from NormData import NormData

# Need to figure out a way to determine where the message is from i.e. gmail, slack, crisp
data = NormData("1", sender, subject, "gmail", data)
print(data)

NormData(
  id: 1
  sender: David Alvarez <dalvarez@optimistinc.com>
  subject: Onboarding
  origin: gmail
  body: Hi Vizcom team,

First off, congrats on your software. I think you guys have something
special here...!

I am reaching out because I have been trying to incorporate the software
into our workflow but I am having trouble shepherding usable results.

I have watched a series of tutorials and the output looks very promising. I
think I just might be missing something because despite all the creative
inputs you guys have incorporated such as palettes, tailored lasso prompts
etc I continue to struggle getting good results.

My workflow typically starts with sketching freehand and from the tutorials
this seems to be the designed workflow but wanted to know if it's maybe my
sketch style that's derailing the process?

In any case, I saw that with some memberships you note onboarding so
thought I would check in to see if any guidance was possible.

Thank you,
DA

  type_of_q: 
  clos

## Labeling The Data using LLM


In [117]:
user_input = str(data)

# more labels needed?
labeler_messages = [
    {
        "role": "system",
        "content": "You are to only label what type of question, a secondary, and tertiary label that may be of relevance, nothing else. "
        "Available Labels = ['payment plans/subscription', 'technical support/bug', 'How to/Instruction', 'Probably Needs Human Intervention']"
        "In this format: 'option1,option2,option3' ",
    },
    {"role": "user", "content": user_input},
]

completion = client.chat.completions.create(
    model="gpt-4o-mini", messages=labeler_messages
)

# multiple labels
label = str(completion.choices[0].message.content).split(",")
# for x in label.split(","):
#     print(x)
data.type_of_q = label
print(data)

NormData(
  id: 1
  sender: David Alvarez <dalvarez@optimistinc.com>
  subject: Onboarding
  origin: gmail
  body: Hi Vizcom team,

First off, congrats on your software. I think you guys have something
special here...!

I am reaching out because I have been trying to incorporate the software
into our workflow but I am having trouble shepherding usable results.

I have watched a series of tutorials and the output looks very promising. I
think I just might be missing something because despite all the creative
inputs you guys have incorporated such as palettes, tailored lasso prompts
etc I continue to struggle getting good results.

My workflow typically starts with sketching freehand and from the tutorials
this seems to be the designed workflow but wanted to know if it's maybe my
sketch style that's derailing the process?

In any case, I saw that with some memberships you note onboarding so
thought I would check in to see if any guidance was possible.

Thank you,
DA

  type_of_q: ['How t

## No Context Response + BS Confidence Score?


In [68]:
import json

In [76]:
user_input = str(data)
context = None

support_bot = [
    {
        "role": "system",
        "content": f"Answer questions with based only off of given context: {context}. Do so politely and professionally with a consistent formatting, "
        "Hello [sender]\n[body]\nThank You,\nVizcom Team"
        f"here is the question type for better reference {data.type_of_q}. Also provide a confidence score based on how accurate you feel your answer is"
        "Only use the context given and conversation, nothing else. Return low confidence scores if not enough context is given"
        "Return a json containing the response and confidence score. Make sure the score is reasonable. Do nothing else."
        "Here is an example json: '{response: 'text', confidence: float/1.0}' ",
    },
    {"role": "user", "content": user_input},
]

completion = client.chat.completions.create(model="gpt-4o-mini", messages=support_bot)
response = str(completion.choices[0].message.content)

pprint(json.loads(response))

{'confidence': 0.85,
 'response': 'Hello David,\n'
             '\n'
             'Thank you for reaching out and for your kind words about our '
             'software! We appreciate your enthusiasm and are here to help you '
             'incorporate it into your workflow successfully.\n'
             '\n'
             'It sounds like you’re on the right path by sketching freehand, '
             'as shown in our tutorials. However, if you’re consistently '
             'struggling to achieve usable results, it might be beneficial for '
             'us to offer personalized guidance. Sometimes, adjustments in '
             'sketching techniques or software settings can make a significant '
             'difference.\n'
             '\n'
             'We would be happy to assist you further with onboarding '
             'guidance. Please let us know your availability for a session, '
             "and we'll arrange a time to walk through your process together.\n"
             '\n'
 

In [128]:
import re


def load_as_string(path: str) -> str:
    with open(path, "r", encoding="utf-8") as f:
        data = f.read()
        return re.sub(r"\s+", " ", data).strip()


ctx1_raw = load_as_string("contexts/documentation.txt")
ctx2_raw = load_as_string("contexts/faqs.txt")

print(ctx1_raw + ctx2_raw)

{ [ "page 1": "Getting started with Render mode", "content": { "description": "In this tutorial, you will learn how to go from sketch to render in just a few seconds with a simple sketch and a text description.", "steps": [ { "step": 1, "title": "Create a new file", "instructions": [ "Choose which layout you want to start your project with.", "If starting in the drawing studio, you can instantly begin sketching on the canvas.", "If you already have a sketch, choose the import option to explore on the infinite canvas." ] }, { "step": 2, "title": "Set up the canvas", "instructions": [ "Create the canvas and start sketching, or upload a previous sketch.", "Learn more about how to import your sketches in the documentation.", "Learn how to draw inside Vizcom in the documentation." ] }, { "step": 3, "title": "Enter a text prompt", "instructions": [ "Open the right-toolbar dropdown (Create tab) and ensure you’re in Render mode.", "Render mode is required for any line drawing or sketch you pla

# Responses with Context?


In [129]:
user_input = "If I downgrade from a paid Professional plan to the free Starter plan mid-billing cycle, what happens to my extra editor seats—are they immediately deactivated or demoted to viewers, and am I eligible for any prorated refund for those unused seats?"
context = ctx1_raw + ctx2_raw

support_bot = [
    {
        "role": "system",
        "content": f"Answer questions with based only off of given context and nothing else (DO NOT GUESS ANSWERS): {context}. Do so politely and professionally with a consistent formatting, "
        "Hello [sender]\n[body]\nThank You,\nVizcom Team"
        f"here is the question type for better reference {data.type_of_q}. Provide a confidence score based on how accurate you feel your answer is, and thought process and citation to context if possible."
        "Only use the context given and conversation, nothing else. Return low confidence scores if not enough context is given"
        "Return a json containing the response and confidence score. Make sure the score is reasonable. Do nothing else."
        'Here is an example json: \'{"response": "text", "confidence": float/1.0, "thought_process/citation": "text"}\' ',
    },
    {"role": "user", "content": user_input},
]

completion = client.chat.completions.create(model="gpt-4o-mini", messages=support_bot)
response = str(completion.choices[0].message.content)
try:
    {pprint(json.loads(response))}
except:
    {print(response)}

{'confidence': 1.0,
 'response': 'If you downgrade from a paid Professional plan to the free '
             'Starter plan mid-billing cycle, any extra editor seats beyond '
             'the limit of two paid seats will be demoted to viewers at the '
             'end of the current billing period. You will not receive a '
             'prorated refund for those unused seats, as the seats will remain '
             'active until the end of the billing cycle.',
 'thought_process/citation': 'This is based on the information provided in the '
                             'context regarding the effects of downgrading '
                             'from a Professional plan to a Starter plan.'}
